# ETL demo

The core concept of the ETL is to operate on the data. 
So you need to understand the data, and know the relationships between them.
And then using scripts to sort them out inside the database, or trigger next progress.

It is a powerful way to get multiple data sources to work together, get the insights or applications we want. 

We will use SQL Server, which is backed by Microsoft as an example to do an ETL demo. 
The goal here will be try to load the data from a csv file to the database.
(In this way you can learn relational database solutions for different vendors, at the same time, SQL Server and PostgreSQL are the two most popular relational database on the market)


## Connect to the SQL Server
first, same as the postgresql connection process, you will need to have a driver to help you connect to the database via python.
You have two options here

- pyodbc
- pymssql

Either way works, we pick one and continue this

In [1]:
import pymssql

In [2]:
# Define your connection parameters
server_name = 'sqlserver'
database_name = 'AdventureworksDWDemo'
username = 'sa'
password = 'YourStrongPassw0rd'

In [3]:
conn = pymssql.connect(server_name, username, password, database_name)

In [4]:
cursor = conn.cursor()

In [5]:
script_parts = [
    "USE AdventureworksDWDemo",
    "CREATE TABLE DimCustomer (CustomerID int PRIMARY KEY IDENTITY, CustomerAltID varchar(50) NOT NULL, CustomerName varchar(256), Gender varchar(20))",
    "CREATE TABLE DimProduct (ProductKey int PRIMARY KEY IDENTITY, ProductAltKey varchar(10) NOT NULL, ProductName varchar(100), ProductActualCost money, ProductSalesCost money)",
    '''
      CREATE TABLE DimStores
    (
        StoreID int PRIMARY KEY IDENTITY,
        StoreAltID varchar(10) NOT NULL,
        StoreName varchar(100),
        StoreLocation varchar(100),
        City varchar(100),
        State varchar(100),
        Country varchar(100)
    )
    ''',
    '''
    CREATE TABLE DimSalesPerson
    (
        SalesPersonID int PRIMARY KEY IDENTITY,
        SalesPersonAltID varchar(10) NOT NULL,
        SalesPersonName varchar(100),
        StoreID int,
        City varchar(100),
        State varchar(100),
        Country varchar(100)
    )
    ''',
    '''
    CREATE TABLE FactProductSales
    (
        TransactionId bigint PRIMARY KEY IDENTITY,
        SalesInvoiceNumber int NOT NULL,
        StoreID int NOT NULL,
        CustomerID int NOT NULL,
        ProductID int NOT NULL,
        SalesPersonID int NOT NULL,
        Quantity float,
        SalesTotalCost money,
        ProductActualCost money,
        Deviation float
    )
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_StoreID FOREIGN KEY (StoreID) REFERENCES DimStores(StoreID)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_CustomerID FOREIGN KEY (CustomerID) REFERENCES DimCustomer(CustomerID)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_ProductKey FOREIGN KEY (ProductID) REFERENCES DimProduct(ProductKey)
    ''',
    '''
    ALTER TABLE FactProductSales ADD CONSTRAINT FK_SalesPersonID FOREIGN KEY (SalesPersonID) REFERENCES DimSalesPerson(SalesPersonID)
    '''
]

In [6]:
for part in script_parts:
    try:
        cursor.execute(part)
        conn.commit()  # Commit changes for DDL statements
    except Exception as e:
        print(f"Error executing SQL script: {e}")
        break  # Stop execution on error

# Close the connection
cursor.close()
conn.close()

until now, we are creating the tables and add the contraints, next we will try to dump the data into the tables.

There are multiple ways to do so, you can select which way best suits you when you doing projects.

- Construct the SQL scripts and execute them as above
- Use pandas (which means other people doing the above step for you)

We will demo the way using pandas

In [7]:
import pandas as pd

In [8]:
from sqlalchemy import create_engine

In [9]:
connection_string = f"mssql+pymssql://{username}:{password}@{server_name}/{database_name}"
engine = create_engine(connection_string)

In [18]:
from pathlib import Path
# data path is current path's parent and then AdventureWorkDWDemo
script_path = Path.cwd().parent
data_path = script_path / "data" / "AdventureWorkDWDemo"

dim_customer_csv = data_path / "DimCustomer.csv"

In [50]:
dim_fact_csv = data_path / "FactProductSales.csv"

In [51]:
fact_df = pd.read_csv(dim_fact_csv, header=None)

In [61]:
fact_df.iloc[1:5,:]

,SalesInvoiceNumber,StoreID,CustomerID,ProductID,SalesPersonID,Quantity,SalesTotalCost,ProductActualCost,Deviation
1,1,1,1,2,1,1,22.5,24.0,1.5
2,1,1,1,3,1,1,42.0,43.5,1.5
3,2,1,2,3,1,1,42.0,43.5,1.5
4,2,1,2,4,1,3,54.0,60.0,6.0


In [53]:
fact_df.columns = ["SalesInvoiceNumber", "StoreID", "CustomerID","ProductID","SalesPersonID","Quantity","SalesTotalCost","ProductActualCost","Deviation"]

In [13]:
CREATE TABLE FactProductSales
    (
        TransactionId bigint PRIMARY KEY IDENTITY,
        SalesInvoiceNumber int NOT NULL,
        StoreID int NOT NULL,
        CustomerID int NOT NULL,
        ProductID int NOT NULL,
        SalesPersonID int NOT NULL,
        Quantity float,
        SalesTotalCost money,
        ProductActualCost money,
        Deviation float
    )

In [62]:
fact_df.to_sql("FactProductSales", con=engine, if_exists="append", index=False)

25

In [15]:
# confirm it via run query 

In [63]:
conn = pymssql.connect(server_name, username, password, database_name)
cursor = conn.cursor()

cursor.execute("SELECT * FROM FactProductSales")

# Fetch all rows
rows = cursor.fetchall()

if rows:
    # Print each row
    for row in rows:
        print(row)

(1, 1, 1, 1, 1, 1, 2.0, Decimal('11.0000'), Decimal('13.0000'), 2.0)
(2, 1, 1, 1, 2, 1, 1.0, Decimal('22.5000'), Decimal('24.0000'), 1.5)
(3, 1, 1, 1, 3, 1, 1.0, Decimal('42.0000'), Decimal('43.5000'), 1.5)
(4, 2, 1, 2, 3, 1, 1.0, Decimal('42.0000'), Decimal('43.5000'), 1.5)
(5, 2, 1, 2, 4, 1, 3.0, Decimal('54.0000'), Decimal('60.0000'), 6.0)
(6, 3, 1, 3, 2, 2, 2.0, Decimal('11.0000'), Decimal('13.0000'), 2.0)
(7, 3, 1, 3, 3, 2, 1.0, Decimal('42.0000'), Decimal('43.5000'), 1.5)
(8, 3, 1, 3, 4, 2, 3.0, Decimal('54.0000'), Decimal('60.0000'), 6.0)
(9, 3, 1, 3, 5, 2, 1.0, Decimal('135.0000'), Decimal('139.0000'), 4.0)
(10, 4, 1, 1, 1, 1, 2.0, Decimal('11.0000'), Decimal('13.0000'), 2.0)
(11, 4, 1, 1, 2, 1, 1.0, Decimal('22.5000'), Decimal('24.0000'), 1.5)
(12, 5, 1, 2, 3, 1, 1.0, Decimal('42.0000'), Decimal('43.5000'), 1.5)
(13, 5, 1, 2, 4, 1, 3.0, Decimal('54.0000'), Decimal('60.0000'), 6.0)
(14, 6, 1, 3, 2, 2, 2.0, Decimal('11.0000'), Decimal('13.0000'), 2.0)
(15, 6, 1, 3, 5, 2, 1.0, De

You can do the rest for the csv files under the AdventureWorkDWDemo folder, and then use this db as the db for the cube creation.